In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

In [2]:
weather_df = pd.read_csv("../Resources/city_data.csv")
weather_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max temp,Wind Speed
0,Vila Franca do Campo,20,PT,1587116354,77,37.72,-25.43,59.00,8.05
1,Punta Arenas,0,CL,1587116354,75,-53.15,-70.92,42.80,10.29
2,Busselton,87,AU,1587116355,32,-33.65,115.33,70.00,9.42
3,Tiksi,59,RU,1587116355,91,71.69,128.87,-2.72,4.79
4,Butaritari,58,KI,1587116355,78,3.07,172.79,83.73,15.30
...,...,...,...,...,...,...,...,...,...
570,Salinópolis,100,BR,1587116517,82,-0.61,-47.36,79.30,3.40
571,Macheng,26,CN,1587116517,38,31.18,115.03,72.19,10.36
572,Lethem,95,GY,1587116518,70,3.38,-59.80,77.00,7.87
573,Natal,40,BR,1587116518,100,-5.79,-35.21,75.20,4.70


In [3]:
# Configure gmaps with API key
gmaps.configure(api_key=g_key)

In [4]:
# Store 'Lat' and 'Lng' into  locations 
locations = weather_df[["Lat", "Lng"]].astype(float)

# Convert Poverty Rate to float and store
# HINT: be sure to handle NaN values
humidity_weight = weather_df["Humidity"].astype(float)

In [5]:
# Create a poverty Heatmap layer
fig = gmaps.figure(zoom_level = 4, center = (41.850033,-87.6500523))

heat_layer = gmaps.heatmap_layer(locations, weights=humidity_weight, 
                                 dissipating=False, max_intensity=300,
                                 point_radius = 4)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

In [6]:
ideal_vaca = weather_df.loc[(weather_df["Humidity"]<=68)\
                            & (weather_df["Max temp"]>=72)\
                            & (weather_df["Max temp"]<=95)\
                            & (weather_df["Wind Speed"]<=3)
                           ]
ideal_vaca["Hotel Name"] = ""

C:\Users\jerin\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [7]:
# find the closest restaurant of each type to coordinates

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "radius": 5000,
    "type": "Lodging",
    "key": g_key,
}
# use iterrows to iterate through pandas dataframe
for index, row in ideal_vaca.iterrows():

    params["location"] = f"{row['Lat']},{row['Lng']}"

    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    try:
        
        ideal_vaca.loc[index, 'Hotel Name'] = results[0]['name']
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

C:\Users\jerin\Anaconda3\lib\site-packages\pandas\core\indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


------------
------------
------------
------------
------------
------------
------------
------------
------------
------------
------------


In [8]:
ideal_vaca

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max temp,Wind Speed,Hotel Name
137,Mpophomeni,0,ZA,1587116394,67,-29.57,30.19,75.00,1.01,Howick
181,Hithadhoo,100,MV,1587116407,68,-0.60,73.08,83.88,2.82,Addu City
192,Richards Bay,0,ZA,1587116411,62,-28.78,32.04,75.42,0.56,Richards Bay
226,Lavumisa,14,SZ,1587116420,50,-27.31,31.89,81.32,0.69,Lavumisa
231,Xam Nua,87,LA,1587116422,43,20.42,104.04,78.21,2.53,Sam Nuea
282,Port Blair,84,IN,1587116232,67,11.67,92.75,84.74,1.61,Port Blair
322,Pekalongan,94,ID,1587116446,68,-6.89,109.68,86.05,2.66,Pekalongan
411,Miandrivazo,0,MG,1587116472,41,-19.52,45.47,88.77,2.86,Miandrivazo
415,Kérouané,43,GN,1587116473,41,9.27,-9.02,86.52,2.77,Kerouane
450,Itanagar,40,IN,1587116484,46,27.10,93.62,91.40,2.24,Itanagar


In [9]:
# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in ideal_vaca.iterrows()]
locations = ideal_vaca[["Lat", "Lng"]]

In [10]:
markers = gmaps.marker_layer(locations)
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))